Есть две идеи:
### 1-ая идея:

- Обучение:
Вытащить из корпуса все positive / negative слова
Векторизовать каждое слово с помощью word2vec
Написать простой классификатор (вектор - класс)

- Тестирование:
На вход поступает слово
Вектурезуем его с помощью word2vec и делаем predict


### 2-ая идея

- Обучение
Подать на word2vec сначала негативные слова, векторизовать, записать в массив. 
Потом подать на word2vec позитивные, векторизовать. 
Получится два облака.

- Тестирование
Далее входные слова сравнивать с каждым из этих двух облаков.
Берем входное слово, сравниваем его с каждым словом из позитивного облака, запиисываем расстояние. 
Потом сравниваем с каждым словом из негативного облака и тоже считаем расстояние. 
Сравниваем результаты по расстояниям по разным облакам.

In [45]:
# -*- coding: utf-8 -*-
from __future__ import division
import re
import random
import pandas as pd
import numpy as np
from lxml import etree

import pymorphy2
import math
from nltk.tokenize import TreebankWordTokenizer
from stop_words import get_stop_words

import gensim

import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn import cross_validation

morph = pymorphy2.MorphAnalyzer()
tokenizer = TreebankWordTokenizer()

RUS_LETTERS = u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [46]:
def parse_xml(filename):
    with open(filename, encoding='utf-8') as f:
        xml = f.read()

    dict = {}
    text = []
    category = []
    sentiment = []
    term = []

    root = etree.fromstring(xml)
    for child in root:
        for aspect in child[3]:
            if aspect.attrib['type'] == 'implicit' and aspect.attrib['sentiment']!= 'both' and aspect.attrib['sentiment']!= 'neutral':
                text.append(child[2].text)
                category.append(aspect.attrib['category'])
                sentiment.append(aspect.attrib['sentiment'])
                term.append(aspect.attrib['term'])

    dict['text'] = text
    dict['category'] = category
    dict['sentiment'] = sentiment
    dict['term'] = term

    
    return dict

In [47]:
text_train = parse_xml('SentiRuEval_rest_markup_train.xml')
text_test = parse_xml('SentiRuEval_rest_markup_test.xml')

In [48]:
# Создаем датафрейм из тестового и тренировочного корпуса

df1 = pd.DataFrame(text_train)
df2 = pd.DataFrame(text_test)

frames = [df1, df2]
df = pd.concat(frames)

In [49]:
df.head()

,category,sentiment,term,text
0,Food,positive,вкусное,"День 8-го марта прошёл, можно и итоги подвести..."
1,Interior,positive,уютный,"День 8-го марта прошёл, можно и итоги подвести..."
2,Food,positive,вкусные,Отмечали в этом ресторане день рождение на пер...
3,Interior,positive,уютно,Отмечали в этом ресторане день рождение на пер...
4,Interior,positive,красиво,Отмечали в этом ресторане день рождение на пер...


In [50]:
df.describe()

,category,sentiment,term,text
count,1367,1367,1367,1367
unique,5,2,505,371
top,Food,positive,вкусно,Отмечали свадьбу 15 марта. Ах-Ах!!! Замечатель...
freq,538,1235,137,14


In [51]:
df.groupby('sentiment').describe()

category    term  \
sentiment                           
negative  count       132     132   
          unique        5     105   
          top        Food  дорого   
          freq         54       5   
positive  count      1235    1235   
          unique        5     402   
          top        Food  вкусно   
          freq        484     137   

                                                               text  
sentiment                                                            
negative  count                                                 132  
          unique                                                 91  
          top     Посещали это заведение 2 раза, ощущения разные...  
          freq                                                    4  
positive  count                                                1235  
          unique                                                353  
          top     Отмечали свадьбу 15 марта. Ах-Ах!!! Замечатель...  
          freq                                                   14

In [52]:
# Делаем датасет сбалансированным
df = pd.concat([df[df['sentiment'] == 'positive'].sample(frac=1)[:150], df[df['sentiment'] == 'negative']]).sample(frac=1)

In [53]:
# Загружаем модель
m = 'web_0_300_20.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
model.init_sims(replace=True)

In [54]:
transit = {'ADJF':'ADJ',
'ADJS' : 'ADJ',
'ADVB' : 'ADV',
'COMP' : 'ADV',
'CONJ' : 'CCONJ',
'GRND' : 'VERB',
'INFN' : 'VERB',
'INTJ' : 'INTJ',
'LATN' : 'X',
'NOUN' : 'NOUN',
'NPRO' : 'PRON',
'NUMB' : 'NUM',
'NUMR' : 'NUM',
'PNCT' : 'PUNCT' ,
'PRCL' : 'PART',
'PRED' : 'ADV',
'PREP' : 'ADP',
'PRTF' : 'ADJ',
'PRTS' : 'VERB',
'ROMN' : 'X',
'SYMB' : 'SYM',
'UNKN' : 'X',
'VERB' : 'VERB'}

robj = re.compile('|'.join(transit.keys()))

def cleanization(text):
    for line in text:
        # 1. Все буквы в нижний регистр
        text_text = text.lower()

        # 2. Удаление всех небукв
        letters_only = ''
        for _c in text_text:
            if _c in RUS_LETTERS:
                letters_only += _c
            else:
                letters_only += ' '

        # 3. Заменяем множественные пробелы
        while '  ' in letters_only:
            letters_only = letters_only.replace('  ', ' ')

        # 4. Токенизация
        word_list = tokenizer.tokenize(letters_only)

        # 5. Лемматизация
        clean_word_list = [morph.parse(word)[0].normal_form for word in word_list]  # лемматизация
    
        # 6. * Удаление стоп-слов + добавление тегов - части речи
        # meaningful_words = [word for word in clean_word_list if word not in get_stop_words('ru')] # стоп-слова
        meaningful_words = [str(word) + '_' + robj.sub(lambda m: transit[m.group(0)], str(morph.parse(word)[0].tag.POS)) for word in clean_word_list]
        return ' '.join(meaningful_words) 

In [55]:
# Проверка работы функции cleanization
cleanization('Я сегодня съела очень много мороженого')

'я_PRON сегодня_ADV съесть_VERB очень_ADV много_ADV мороженое_NOUN'

In [56]:
from __future__ import division
def mean(a):
    return sum(a) / len(a)


def word2vec_mean(text):
    """Усредняет вектор слов."""
    arr = []
    clean_text = cleanization(text)
    # для каждого слова в тексте выводим его вектор
    for word in clean_text.split(' '):
        # есть ли слово в модели? Может быть, и нет
        if word in model:
            arr.append(model[word])
    if len(list(map(mean, zip(*arr)))) != 0:
        return list(map(mean, zip(*arr)))
    else:
        return [0 for i in range(0, 300)]



class FunctionFeaturizer(TransformerMixin):
    """ Для создания своего вектора я использовала усредненную векторизацию с помощью word2vec"""
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        fvs = []
        # fvs = word2vec_mean(X)  # если подавать по словам, а не датафрейм
        for datum in X:
            fv = word2vec_mean(datum)
            fvs.append(fv)
        return np.array(fvs)

In [57]:
# Проверка векторизатора
df1 = pd.DataFrame({'text':['хороший', 'очень хороший'], 'class':['1', '0']})
w2v_featurizer = FunctionFeaturizer() 
x = w2v_featurizer.transform(df1.text)
x

array([[  7.33737042e-03,  -2.96452902e-02,   6.50390610e-02,
          3.19109261e-02,   4.67816442e-02,  -6.75016195e-02,
          1.34420572e-02,  -1.00418180e-01,   6.53345138e-02,
          2.46328283e-02,  -7.88142625e-03,  -7.19515756e-02,
         -3.37976255e-02,   2.32609529e-02,   5.84121747e-03,
         -1.20705580e-02,  -3.55095118e-02,   5.70581928e-02,
         -5.28854914e-02,   6.36451971e-03,   4.01164368e-02,
         -4.82101887e-02,  -5.52522391e-02,   1.19763343e-02,
          5.54701425e-02,  -1.48677127e-02,  -1.01240285e-01,
          4.72908318e-02,   4.67002094e-02,   6.18545637e-02,
         -1.20627820e-01,  -9.26931016e-03,   3.15161943e-02,
          2.23013968e-03,  -3.23524722e-03,   3.60459499e-02,
          2.32331306e-02,   4.34190817e-02,  -6.47412986e-02,
          1.71083870e-04,   2.29924582e-02,  -2.39938311e-02,
          3.65550630e-02,   5.90851046e-02,   4.88272235e-02,
         -6.65662438e-02,   1.19906226e-02,   6.42403737e-02,
        

In [58]:
# Делим корпус на тестовый и тернировочный
X_train, X_test, y_train, y_test = train_test_split(df['term'], df['sentiment'], test_size=0.1)

In [59]:
X_train.head()

258       претензионная
430    профессиональное
613              быстро
522           не вкусно
139    доброжелательный
Name: term, dtype: object

In [60]:
y_train.head()

258    negative
430    positive
613    positive
522    negative
139    positive
Name: sentiment, dtype: object

In [61]:
X_train.describe()

count        253
unique       180
top       вкусно
freq          14
Name: term, dtype: object

In [62]:
def do_smth_with_model(data_train, class_train, data_test, class_test, steps):
    print('\nModel train')
    pipeline = Pipeline(steps=steps)

    cv_results = cross_val_score(pipeline,
                                 data_train,
                                 class_train,
                                 cv=10,
                                 scoring='accuracy',
                                )
    print(cv_results.mean(), cv_results.std())

    pipeline.fit(data_train, class_train)
    class_predicted = pipeline.predict(data_test)
    print(class_predicted)

    print(classification_report(class_test, class_predicted ))

    return pipeline, class_predicted

In [63]:
w2v_featurizer = FunctionFeaturizer()  # создание своего векторизатора

In [64]:
# Свой векторизатор
print('\nCustom Transformer')
lr_pipeline, label_predicted = do_smth_with_model(X_train, y_train,
                                               X_test, y_test, 
                                               steps=[('custom', w2v_featurizer),
                                                      ('classifier', LogisticRegression())])


Custom Transformer

Model train
0.878076923077 0.0523608327353
['negative' 'negative' 'positive' 'negative' 'negative' 'positive'
 'negative' 'negative' 'negative' 'negative' 'positive' 'negative'
 'positive' 'positive' 'positive' 'negative' 'negative' 'negative'
 'positive' 'negative' 'positive' 'negative' 'negative' 'negative'
 'negative' 'negative' 'positive' 'positive' 'negative']
             precision    recall  f1-score   support

   negative       0.74      1.00      0.85        14
   positive       1.00      0.67      0.80        15

avg / total       0.87      0.83      0.82        29



In [65]:
# Свой векторизатор
print('\nCustom Transformer')
etx_pipeline, label_predicted = do_smth_with_model(X_train, y_train,
                                               X_test, y_test, 
                                               steps=[('custom', w2v_featurizer),
                                                      ('classifier', ExtraTreesClassifier())])


Custom Transformer

Model train
0.847 0.0591039879688
['positive' 'negative' 'positive' 'negative' 'negative' 'positive'
 'negative' 'positive' 'positive' 'negative' 'positive' 'negative'
 'positive' 'negative' 'positive' 'negative' 'negative' 'negative'
 'positive' 'positive' 'positive' 'negative' 'negative' 'negative'
 'negative' 'negative' 'positive' 'positive' 'negative']
             precision    recall  f1-score   support

   negative       0.88      1.00      0.93        14
   positive       1.00      0.87      0.93        15

avg / total       0.94      0.93      0.93        29



In [66]:
# Word2Vec + RandomForestClassifier
print('\nCustom Transformer + RandomForestClassifier')
rf_pipeline, label_predicted = do_smth_with_model(X_train, y_train,
												   X_test, y_test,
												   steps=[('custom', w2v_featurizer),
														  ('classifier', RandomForestClassifier())])



Custom Transformer + RandomForestClassifier

Model train
0.843461538462 0.0731885786673
['negative' 'positive' 'positive' 'negative' 'negative' 'positive'
 'negative' 'positive' 'negative' 'negative' 'positive' 'negative'
 'positive' 'positive' 'positive' 'negative' 'negative' 'negative'
 'positive' 'positive' 'positive' 'negative' 'negative' 'negative'
 'negative' 'negative' 'positive' 'positive' 'negative']
             precision    recall  f1-score   support

   negative       0.81      0.93      0.87        14
   positive       0.92      0.80      0.86        15

avg / total       0.87      0.86      0.86        29



In [67]:
# Word2Vec + DecisionTreeClassifier
print('\nCustom Transformer + DecisionTreeClassifier')
dt_pipeline, label_predicted = do_smth_with_model(X_train, y_train,
												   X_test, y_test,
												   steps=[('custom', w2v_featurizer),
														  ('classifier', DecisionTreeClassifier())])


Custom Transformer + DecisionTreeClassifier

Model train
0.842858974359 0.0635821968744
['positive' 'negative' 'positive' 'negative' 'positive' 'positive'
 'negative' 'positive' 'positive' 'negative' 'positive' 'negative'
 'positive' 'positive' 'positive' 'negative' 'positive' 'negative'
 'positive' 'positive' 'positive' 'positive' 'positive' 'negative'
 'negative' 'negative' 'positive' 'positive' 'negative']
             precision    recall  f1-score   support

   negative       1.00      0.79      0.88        14
   positive       0.83      1.00      0.91        15

avg / total       0.91      0.90      0.90        29



## Проверка работы модели на наших тестовых коллокациях

In [68]:
df1 = pd.DataFrame({'text':['отличный выбор', 'не советуем', 'очень советуем', 'очень дорого', 'выше всяких похвал', 'в общем прекрасно', 'нам все понравилось']})
# , 'в целом ничего', 'отвратительный', 'быстро', 'очень плохое обслуживание', 'отличное меню', 'хороший', 'вкусный', 'замечательный', 'приятный', 'красивый', 'отличный'

In [69]:
df1.head()

,text
0,отличный выбор
1,не советуем
2,очень советуем
3,очень дорого
4,выше всяких похвал


In [70]:
x = w2v_featurizer.transform(df1.text)  # векторизуем наши слова

In [71]:
x

array([[-0.02267521, -0.02004785, -0.0533559 , ..., -0.00224797,
         0.07374597, -0.04249991],
       [-0.01177259, -0.06730742,  0.06288135, ..., -0.01996527,
         0.11107853, -0.05057059],
       [-0.00887533, -0.04581857,  0.06392314, ..., -0.02286776,
         0.04544817, -0.03894411],
       ..., 
       [ 0.01659136, -0.02731018, -0.00448782, ...,  0.04818018,
         0.00575948, -0.09879748],
       [ 0.04194519, -0.08253068, -0.0113095 , ..., -0.06132992,
         0.02059018, -0.05131358],
       [-0.03626946,  0.05354116,  0.04102799, ..., -0.03269978,
         0.08353947, -0.05340419]])

In [72]:
lr_model = LogisticRegression()
lr_model.fit(w2v_featurizer.transform(X_train), y_train)
cv_results = cross_val_score(lr_model, w2v_featurizer.transform(X_train), y_train, cv=10, scoring='accuracy')
print(cv_results.mean(), cv_results.std())
lr_model.predict(x)

0.878076923077 0.0523608327353


array(['positive', 'negative', 'negative', 'negative', 'negative',
       'negative', 'positive'], dtype=object)

In [73]:
# Проверка работы модели на наших тестовых коллокациях
def predictor(collocations_array, pipeline):
    arr = []
    df1 = pd.DataFrame({'text': collocations_array})
    for i in df1.text:
        arr.append(i)
    с = 0
    for i in pipeline.predict(df1.text):
        print(arr[с], ':', i)
        с += 1


collocations_array = ['отвратительный', 'быстро', 'очень плохое обслуживание', 'отличное меню']
predictor(collocations_array, etx_pipeline)


отвратительный : negative
быстро : positive
очень плохое обслуживание : negative
отличное меню : negative


In [74]:
collocations_array = ['отличный выбор', 'не советуем', 'очень советуем', 'очень дорого', 'выше всяких похвал', 'в общем прекрасно', 'нам все понравилось', 'в целом ничего', 'отвратительный', 'быстро', 'очень плохое обслуживание', 'отличное меню', 'хороший', 'вкусный', 'замечательный', 'приятный', 'красивый', 'отличный']


# Ввести правильные ответы
true = {'отличный выбор': 'positive',
'не советуем': 'negative',
'очень советуем': 'positive',
'очень дорого': 'negative',
'выше всяких похвал': 'positive',
'в общем прекрасно': 'positive',
'нам все понравилось': 'positive',
'в целом ничего': 'positive',
'отвратительный': 'negative',
'быстро': 'positive',
'очень плохое обслуживание': 'negative',
'отличное меню' : 'positive',
'хороший' : 'positive',
'вкусный' : 'positive',
'замечательный' : 'positive',
'приятный' : 'positive',
'красивый' : 'positive',
'отличный' : 'positive'}



# Проверка работы модели на наших тестовых коллокациях
def predictor(collocations_array, pipeline):
	mistakes = 0
	arr = []
	df1 = pd.DataFrame({'text': collocations_array})
	for i in df1.text:
		arr.append(i)
	с = 0
	for i in pipeline.predict(df1.text):
		print(arr[с], ':', i)
		if true[arr[с]] != i:
			mistakes += 1
		с += 1
	print(mistakes)


# ВВЕДИТЕ СЛОВА, КОТОРЫЕ ХОТИТЕ ПРОВЕРИТЬ
predictor(collocations_array, etx_pipeline)
print('_'*30)
predictor(collocations_array, lr_pipeline)
print('_'*30)
predictor(collocations_array, rf_pipeline)
print('_'*30)
predictor(collocations_array, dt_pipeline)

отличный выбор : negative
не советуем : negative
очень советуем : negative
очень дорого : negative
выше всяких похвал : negative
в общем прекрасно : negative
нам все понравилось : negative
в целом ничего : negative
отвратительный : negative
быстро : positive
очень плохое обслуживание : negative
отличное меню : negative
хороший : positive
вкусный : positive
замечательный : positive
приятный : positive
красивый : positive
отличный : negative
8
______________________________
отличный выбор : positive
не советуем : negative
очень советуем : negative
очень дорого : negative
выше всяких похвал : negative
в общем прекрасно : negative
нам все понравилось : positive
в целом ничего : negative
отвратительный : negative
быстро : positive
очень плохое обслуживание : negative
отличное меню : positive
хороший : positive
вкусный : positive
замечательный : positive
приятный : positive
красивый : positive
отличный : positive
4
______________________________
отличный выбор : positive
не советуем : negati